In [1]:
import json
from PIL import Image
import re
import random
random.seed(777)

### train_set 만들기

In [2]:
%cd C:\Users\na062\Desktop\25000 # 데이터 있는 폴더로 이동

C:\Users\na062\Desktop\25000


In [3]:
def caption_shuffle(caption):
    for annotation in caption:
        random.shuffle(annotation['label'])

In [4]:
def caption_split(annotation):
    caption_set = []
    caption_set.append({'image':annotation['image'],'label':annotation['label'][0]})
    for i in range(1,5):
        imagename = annotation['image'].split('.jpg')[0]+f'-{i}.jpg'
        caption_set.append({'image':imagename,'label':annotation['label'][i]})
    return caption_set

In [5]:
def augment_captions(captions):
    augment_captions=[]
    for annotation in captions:
        augment_captions+=caption_split(annotation)
    return augment_captions

In [6]:
def make_5_captions(caption,ids,num,id_of_captions,orgin_captions):
    origin = re.sub('image','',caption['image'][:-4].split('-')[0])
    number = re.sub('image','',caption['image'][:-4].split('-')[-1]) # ??? 
    # 원래 이미지에 관한 캡션으로 접근하여 5개의 캡션을 받아온다. 
    captionlist= captions[id_of_captions.index(origin)]['label']
    caption_5 = []
    for caption_origin in captionlist:
        caption_5.append({'image_id':ids+1,'caption':caption_origin,'id':num+1})
        num+=1
    ids+=1
    return caption_5


In [7]:
def make_gt(captions,id_of_captions,origin_captions):
    annotations=[]
    images = []
    ids=0
    num=0
    for caption in captions:
        annotations+=make_5_captions(caption,ids,num,id_of_captions,origin_captions)
        images.append({'id':ids+1})
        ids+=1
        num+=5
    gt_captions ={
        'annotations': annotations,
        'images': images
    }
    return gt_captions

In [8]:
with open("caption5_augmented.json",'r') as f:
    captions = json.load(f)

caption_shuffle(captions) # label 임의대로 섞기
match_captions = augment_captions(captions) #train처럼 (1:1)로 만들기 
random.shuffle(match_captions) #  셔플 

train_captions = match_captions[:int(0.8*25000)]
val_captions = match_captions[int(0.8*25000):] # 5000개 *5개 

id_of_captions = [re.sub('image','',caption['image'][:-4]) for caption in captions] # 리스트 생성 
gt_captions = make_gt(val_captions,id_of_captions,captions)

In [9]:
with open ('shuffled_captions.json','w') as f:
    json.dump(match_captions,f)

with open('gt_captions.json','w') as f:
    json.dump(gt_captions,f)